In [1]:
import os 
from dotenv import load_dotenv
load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
llm_config = {"model": "gpt-4o-mini"}

In [2]:
task = """
Write a Concise  but engaging blogpost about the fear of losing a love. Make sure the blog post is
withinn 100 words.
"""

## Create a writer agent¶

In [4]:
import autogen
writer = autogen.AssistantAgent(
    name = "writer",
    system_message = "You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config = llm_config,
)

In [5]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [6]:
print(reply)

**The Fear of Losing Love: A Universal Struggle**

Love is a beautiful journey, yet it often comes with a haunting fear—the fear of losing it. This anxiety can stem from past heartaches, insecurities, or the fragility of human connections. As we grow closer to someone, the thought of separation can become overwhelming. It’s essential to recognize this fear as a natural part of intimacy rather than a signal to withdraw. Embracing vulnerability, communicating openly, and nurturing the relationship can transform our fears into deeper bonds. After all, love’s true strength lies not in possession but in the courage to cherish each moment together.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [7]:
critic = autogen.AssistantAgent(
    name = "critic",
    system_message = "You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
    llm_config = llm_config,
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
)

In [8]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

critic (to writer):


Write a Concise  but engaging blogpost about the fear of losing a love. Make sure the blog post is
withinn 100 words.


--------------------------------------------------------------------------------
writer (to critic):

**The Fear of Losing a Love**

Losing someone you love is a haunting thought that creeps into your mind when you least expect it. This fear often stems from the vulnerability that love brings; the deeper the connection, the greater the risk. Each moment spent together is tinged with the uncertainty of tomorrow. We cling to memories, terrified that they might fade into mere echoes of the past. Yet, this fear can also deepen our appreciation for love itself. Embracing the fragility of relationships can inspire us to cherish every shared smile, laugh, and tear, transforming fear into a celebration of love’s precious moments.

--------------------------------------------------------------------------------
critic (to writer):

Your blog post captures

## Nested chat

In [20]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [21]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [22]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [23]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [24]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [25]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

In [26]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

critic (to writer):


Write a Concise  but engaging blogpost about the fear of losing a love. Make sure the blog post is
withinn 100 words.


--------------------------------------------------------------------------------
writer (to critic):

**The Fear of Losing a Love**

Losing someone you love is a haunting thought that creeps into your mind when you least expect it. This fear often stems from the vulnerability that love brings; the deeper the connection, the greater the risk. Each moment spent together is tinged with the uncertainty of tomorrow. We cling to memories, terrified that they might fade into mere echoes of the past. Yet, this fear can also deepen our appreciation for love itself. Embracing the fragility of relationships can inspire us to cherish every shared smile, laugh, and tear, transforming fear into a celebration of love’s precious moments.

--------------------------------------------------------------------------------

*******************************************

/home/faizrazadec/anaconda3/lib/python3.12/site-packages/autogen/agentchat/conversable_agent.py:1277: UserWarning: Cannot extract summary using reflection_with_llm: Error code: 400 - {'error': {'message': "Invalid 'messages[1].name': string does not match pattern. Expected a string that matches the pattern '^[a-zA-Z0-9_-]+$'.", 'type': 'invalid_request_error', 'param': 'messages[1].name', 'code': 'invalid_value'}}. Using an empty str as summary.
  warnings.warn(


Legal Reviewer (to critic):

As a legal reviewer, here are my suggestions regarding the content titled "The Fear of Losing a Love":

- **Copyright Considerations**: Ensure that the text is original and doesn't infringe on the copyrights of existing literary works. If using quotes or paraphrasing, proper attribution is essential.
- **Emotional Impact**: While the content discusses sensitive emotional themes, consider adding a disclaimer advising readers who may be struggling with similar fears to seek professional support, to mitigate potential liability.
- **Disclaimers**: If the content is intended for a public audience, consider including a general disclaimer that it reflects personal opinions and does not constitute professional advice.



--------------------------------------------------------------------------------

********************************************************************************
Starting a new chat....

***********************************************************

In [27]:
print(res.summary)

**The Fear of Losing Love**

The fear of losing a loved one can be overwhelming, lurking in the corners of our hearts. It often arises from the deep vulnerability love brings; the closer we get to someone, the more we fear their absence. This anxiety can manifest in constant worry or reluctance to express feelings, often leading to misunderstandings and distance. 

However, embracing this fear might just deepen our connections. Acknowledging its presence can inspire us to cherish every moment, making each laugh, conversation, and hug more meaningful. Instead of letting fear rule us, let it remind us to love fiercely and fully while we can.


In [29]:
import pprint
pprint.pprint(res.cost)

{'usage_excluding_cached_inference': {'gpt-4o-mini-2024-07-18': {'completion_tokens': 748,
                                                                 'cost': 0.0006177,
                                                                 'prompt_tokens': 1126,
                                                                 'total_tokens': 1874},
                                      'total_cost': 0.0006177},
 'usage_including_cached_inference': {'gpt-4o-mini-2024-07-18': {'completion_tokens': 1129,
                                                                 'cost': 0.0008867999999999999,
                                                                 'prompt_tokens': 1396,
                                                                 'total_tokens': 2525},
                                      'total_cost': 0.0008867999999999999}}
